# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, make_scorer, fbeta_score, f1_score
from sklearn.model_selection import GridSearchCV
from sqlalchemy import create_engine
from scipy.stats.mstats import gmean
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.base import BaseEstimator,TransformerMixin
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM df1', con = engine)
X = df [['id', 'message', 'original','genre']]
Y = df [['electricity', 'tools', 'hospitals','missing_people','aid_related', 'medical_help', 'medical_products','related', 'request', 'offer', 'search_and_rescue', 'security', 'military', 'child_alone',
 'water', 'food', 'shelter', 'clothing', 'money', 'other_aid', 'infrastructure_related', 'transport',
 'buildings', 'refugees', 'death', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
 'storm', 'other_weather','direct_report','fire', 'earthquake', 'cold']]
Y = df.iloc[:,4:]
X = df['message']


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    '''This is a tokenization function for our text. 
    The input is our text and output is tokenized text'''
    found_urls = re.findall(url_regex, text)
    for url in found_urls:
        text = text.replace(url, "urlplaceholder")
        

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def pipeline():
    '''This is a function for our text. 
    The input is our text and output is tokenized text'''
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf',MultiOutputClassifier(RandomForestClassifier())),
        ])
    return pipeline

In [5]:
def show_results(y_test, y_pred):
    '''This is a function for showing the results 
    The input is the y test and y pred and output is result
    We involve the concept of confusion matrix to acheieve this
    '''
    
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
X_train.head()

13707    In the immediate aftermath of natural disaster...
532      Good Morning I am in Haiti : I need my father ...
4282     I thank you for helping .. we have rice but we...
20460    More than a decade since the first cases of th...
662      Please put a prepaid card ( minutes ) for me. ...
Name: message, dtype: object

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# train classifier
model=[]
model=pipeline()
model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
Y_pred = model.predict(X_test)
category_names = list(Y)
 # Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

Category: related 
              precision    recall  f1-score   support

          0       0.61      0.36      0.45      1176
          1       0.83      0.93      0.88      4022
          2       0.69      0.24      0.35        46

avg / total       0.78      0.80      0.78      5244

Accuracy of                   related: 0.80
Category: request 
              precision    recall  f1-score   support

          0       0.88      0.98      0.93      4338
          1       0.84      0.38      0.53       906

avg / total       0.88      0.88      0.86      5244

Accuracy of                   request: 0.88
Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

          0       0.71      0.88  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
# Calculation the proportion of each column that have label == 1
Y.sum()/len(Y)

related                   0.773650
request                   0.170659
offer                     0.004501
aid_related               0.414251
medical_help              0.079493
medical_products          0.050084
search_and_rescue         0.027617
security                  0.017966
military                  0.032804
child_alone               0.000000
water                     0.063778
food                      0.111497
shelter                   0.088267
clothing                  0.015449
money                     0.023039
missing_people            0.011367
refugees                  0.033377
death                     0.045545
other_aid                 0.131446
infrastructure_related    0.065037
transport                 0.045812
buildings                 0.050847
electricity               0.020293
tools                     0.006065
hospitals                 0.010795
shops                     0.004577
aid_centers               0.011787
other_infrastructure      0.043904
weather_related     

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
def model_build():
    '''
    Building an Machine Learning pipeline using random forest, ifidf, and gridsearch
    Here, there would not be any inpupt
    The output would be the results of gridsearchcv
    '''
    pipe2 = Pipeline([
                      ('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

    param2 = {
             'clf__estimator__n_estimators': [50, 100],
             'clf__estimator__min_samples_split': [2, 3, 4],
             }
    cv = GridSearchCV(pipe2, param_grid=param2)
    
    return cv

In [ ]:
model=model_build()
model.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred = model.predict(X_test)
#Showing the accuracy, precision, and recall of the tuned model.    
# Calculating the accuracy for each of them.
for i in range(len(category_names)):
    print("The Category is:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('The accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

In [ ]:
def fscore_multiop(y_true,y_pred,beta=1):
    '''
    This function is for calculating f scores for multiple outputs
    This function takes three inputs. Namely, y_true,y_pred,beta=1
    The output will be fscores for multiple outputs    
    '''
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_npy = np.asarray(score_list)
    f1score_npy = f1score_npy[f1score_npy<1]
    f1score = gmean(f1score_npy)
    return  f1score

In [ ]:
multiple_f1score = fscore_multiop(Y_test,Y_pred, beta = 1)
overall_accuracy = (Y_pred == Y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score {0:.2f}%\n'.format(multiple_f1score*100))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

* Here I used Adaboost and Gradient Boosting

In [ ]:
def model_build():
    '''
    Building an Machine Learning pipeline using random forest, ifidf, and gridsearch
    Here, there would not be any inpupt
    The output would be the results of gridsearchcv
   
    '''
    pipeline2 = Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                        ])

    parameters2 = {
                  'clf__estimator__n_estimators': [50, 100],
                  'clf__estimator__learning_rate': [0.01,0.1],
                 }
    
    cv = GridSearchCV(pipeline2, param_grid=parameters2)
    
    return cv

In [ ]:
Y_pred = model.predict(X_test)
    
    # Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

In [ ]:
#Calculating F1 score
multi_f1 = fscore_multiop(Y_test,Y_pred, beta = 1)

#calculating overall accuracy for the model
overall_accuracy = (Y_pred == Y_test).mean().mean()

print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))
print('F1 score {0:.2f}%\n'.format(multi_f1*100))

### 9. Export your model as a pickle file

In [ ]:
#Export your model as a pickle file
#Here, the filename is final_model.sav
import pickle
pickle.dump(model, open('final_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.